## 5.1 합성곱 신경망(컨브넷) 소개

간단한 컨브넷 예제   
기본적인 컨브넷이어도 완전 연결된 모델의 성능을 훨씬 앞지를 것

#### 5.1.1 합성곱 연산

Dense 층: 입력 특성 공간에 있는 전역 패턴 학습   
합성곱 층: 지역 패턴 학습 (이미지일 때 작은 2D 윈도우로 입력에서 패턴을 찾음)

- 학습된 패턴은 평행 이동 불변성을 가짐   
        완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 함
        컨브넷은 이미지 효율적으로 처리하게 만들어 줌 (적은 수의 훈련 샘플을 사용해서 일반화 능력 가진 표현을 학습 가능함)
- 컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있음   
        첫 번째 합성곱이 에지 같은 작은 지역 패턴 학습, 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴 학습   
        -> 매우 복잡하고 추상적인 시각적 개념 효과적으로 학습 가능함

합성곱 연산은 특성 맵(3D 텐서)에 적용됨   
2개의 공간축(높이, 너비), 깊이(채널)축으로 구성됨  (RGB 이미지는 3개의 컬러 채널 -> 축의 차원이 3)   
입력 특성 맵에서 작은 패치들 추출, 모든 패치에 같은 변환 적용해 출력 특성 맵을 만듬

출력 텐서(높이, 너비 가진 3D 텐서)의 깊이는 층의 매개변수로 결정됨 -> 깊이 축의 채널은 **필터**를 의미함

(28, 28, 1) 크기의 특성 맵 입력 -> (26, 26, 32) 크기의 특성 맵 출력   
여기서 입력에 대한 32개의 필터를 적용한 것   
32개의 출력 채널 각각은 26x26 크기의 배열 값을 가짐 (입력에 대한 필터의 **응답 맵**   


합성곱 정의에 필요한 2개의 파라미터
- 입력으로부터 뽑아낼 패치의 크기
        보통 3x3, 5x5 크기 사용
- 특성 맵의 출력 깊이
        합성곱으로 계산할 필터 수
        
```
Conv2D(output_depth, (window_height, window_width))
```

합성곱은 3D 입력 특성 맵 위를 3x3 또는 5x5 크기의 윈도우가 슬라이딩 하면서 모든 위치에서 3D 특성 패치 추출하는 방식으로 작동함   
3D 패치는 (output_depth, ) 크기의 1D 벡터로 변환됨 (합성곱 커널 통해 변환)   
변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성됨   
출력 특성 맵의 공간상 위치는 입력 특성 맵의 같은 위치에 대응됨

출력 높이와 너비는 입력 높이와 너비와는 다를 수 있음
- 경계 문제
        입력과 동일한 높이, 너비 가진 출력 특성 맵 얻고 싶으면 패딩 사용할 수 있음
        Conv2D 층에서 padding 매개변수로 설정 가능 (기본값은 valid(패딩 사용 x))
- 스트라이드 사용 여부에 따라 다름
        스트라이드: 두 번의 연속적인 윈도우 사이의 거리
        기본값은 1

#### 5.1.2 최대 풀링 연산

최대 풀링의 역할: 강제적으로 특성 맵 다운샘플링 하는 것

입력 특성 맵에서 윈도우에 맞는 패치 추출하고 각 채널별 최댓값 출력함 (최댓값 추출 연산 사용)   
보통 2x2 윈도우와 스트라이드 2를 사용해 특성 맵을 절반 크기로 다운샘플링함

다운샘플링하는 이유: 처리할 특성 맵의 가중치 개수 줄이기 위해서   
연속적인 합성곱 층이 점점 커진 윈도우 통해 바라보도록 만들어 필터의 공간적 계층 구조 구성함

## 5.2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기

#### 5.2.1 작은 데이터셋 문제에서 딥러닝의 타당성

딥러닝 모델은 조금씩 변경해 다른 문제에 재사용 가능함

#### 5.2.2 데이터 내려받기


In [8]:
#훈련, 검증, 테스트 폴더로 이미지 복사하기

import os, shutil

original_dataset_dir = './datasets/cats_and_dogs/train' #원본 데이터셋 압축 해제한 디렉터리 경로

base_dir= './datasets/cats_and_dogs_small' #소규모 데이터셋 저장할 디렉터리
#os.mkdir(base_dir)

#훈련, 검증, 테스트 분할 위한 디렉터리
train_dir = os.path.join(base_dir, 'train')
#os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
#os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
#os.mkdir(test_dir)

#훈련용 고양이, 강아지 사진 디렉터리
train_cats_dir = os.path.join(train_dir, 'cats')
#os.mkdir(train_cats_dir)

train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)

#검증용 고양이, 강아지 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)

validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)

#테스트용 고양이, 강아지 사진 디렉터리
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)

test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

#처음 1000개의 고양이 이미지를 train_cats_dir에 복사
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
    
#다음 500개의 고양이 이미지를 validation_cats_dir에 복사    
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src,dst)
    
#다음 500개의 고양이 이미지를 test_cats_dir에 복사
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src,dst)
    
#처음 1000개의 강아지 이미지를 train_cats_dir에 복사
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
#다음 500개의 강아지 이미지를 validation_cats_dir에 복사    
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src,dst)
    
#다음 500개의 강아지 이미지를 test_cats_dir에 복사
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src,dst)
        
        
#잘 됐는지 사진 개수 카운트로 확인
print('훈련용 고양이 이미지 전체 개수:', len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:', len(os.listdir(train_dogs_dir)))

print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_cats_dir)))
print('검증용 강아지 이미지 전체 개수:', len(os.listdir(validation_dogs_dir)))

print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_cats_dir)))
print('테스트용 강아지 이미지 전체 개수:', len(os.listdir(test_dogs_dir)))



훈련용 고양이 이미지 전체 개수: 1000
훈련용 강아지 이미지 전체 개수: 1000
검증용 강아지 이미지 전체 개수: 500
검증용 강아지 이미지 전체 개수: 500
테스트용 강아지 이미지 전체 개수: 500
테스트용 강아지 이미지 전체 개수: 500


#### 5.2.3 네트워크 구성하기

Conv2D(relu 활성화 함수 사용), MaxPooling2D 층을 번갈아 쌓은 컨브넷 만들기   
150x150 크기의 입력으로 시작해서 Flatten 층 이전에 7x7 크기의 특성 맵으로 줄어듬

특성 맵의 깊이는 네트워크에서 점진적으로 증가(32 -> 128), 크기는 감소(150x150 -> 7x7)

이진 분류 문제라서 네트워크는 하나의 유닛(크기가 1인 Dense 층), sigmoid 활성화 함수로 끝남

In [9]:
#강아지 vs 고양이 분류를 위한 소규모 컨브넷 만들기

from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)      

In [11]:
#모델 훈련 설정

from keras import optimizers

model.compile(loss='binary_crossentropy', 
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

#### 5.2.4 데이터 전처리

데이터가 JPEG 파일로 되어있어서 네트워크에 주입하려면 과정이 필요함
1. 사진 파일을 읽음
2. JPEG 콘텐츠를 RGB 픽셀 값으로 디코딩
3. 부동 소수 타입의 텐서로 변환
4. 픽셀 값(0~255)의 스케일을 [0,1] 사이로 조정

keras.preprocessing.image에 이미지 처리 위한 헬퍼 도구들 있음   
ImageDataGenerator 클래스는 디스크에 있는 이미지 파일을 전처리된 배치 텐서로 자동으로 바꿔 주는 파이썬 제너레이터를 만들어 줌

In [12]:
#ImageDataGenerator 사용해 디렉터리에서 이미지 읽기

from keras.preprocessing.image import ImageDataGenerator

#모든 이미지를 1/255로 스케일 조정
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150), #모든 이미지를 150x150 크기로 바꿈
                                                   batch_size=20,
                                                   class_mode='binary') #binary_crossentropy 손실 사용해서 이진 레이블 필요함

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       target_size=(150,150),
                                                       batch_size=20,
                                                       class_mode='binary')




Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [13]:
#배치 제너레이터 사용해 모델 훈련

history = model.fit_generator(train_generator,
                             steps_per_epoch=100,
                             epochs=30,
                             validation_data = validation_generator,
                             validation_steps=50)

#모델 저장
model.save('cats_and_dogs_small_1.h5')

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/30
100/100 [==============================] - 38s 383ms/step - loss: 0.6907 - acc: 0.5320 - val_loss: 0.6850 - val_acc: 0.5330
Epoch 2/30
100/100 [==============================] - 44s 438ms/step - loss: 0.6606 - acc: 0.6130 - val_loss: 0.6668 - val_acc: 0.5940
Epoch 3/30
100/100 [==============================] - 46s 457ms/step - loss: 0.6274 - acc: 0.6590 - val_loss: 0.6298 - val_acc: 0.6690
Epoch 4/30
100/100 [==============================] - 46s 458ms/step - loss: 0.5876 - acc: 0.7080 - val_loss: 0.6372 - val_acc: 0.6390
Epoch 5/30
100/100 [==============================] - 46s 456ms/step - loss: 0.5419 - acc: 0.7310 - val_loss: 0.5941 - val_acc: 0.6850
Epoch 6/30
100/100 [==============================] - 44s 443ms/step - loss: 0.5109 - acc: 0.7500 - val_loss: 0.6250 - val_acc: 0.6620
Epoch 7/30
100/100 [==============================] - 44s 442ms/step - loss: 0.4822 - acc: 0.7630 - val_loss: 0.60

In [ ]:
#훈련의 정확도와 손실 그래프 그리기

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label = 'Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label = 'Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

#과대적합의 특성 보여줌

#### 5.2.5 데이터 증식 사용하기

기존 훈련 샘플로부터 더 많은 훈련 데이터 생성하는 방법

In [14]:
#ImageDataCenerator 사용해 데이터 증식 설정하기

datagen = ImageDataGeneratorGeneratorDataGenerator(rotation_range=20, #랜덤하게 사진 회전시킬 각도 범위(0~180)
                            width_shift_range=0.1, #사진을 수평/수직으로 랜덤하게 평행 이동시킬 범위(전체 높이,너비에 대한 비율)
                            height_shift_range=0.1,
                            shear_range=0.1, #랜덤하게 전단 변환 적용할 각도 범위
                            zoom_range=0.1, #랜덤하게 사진 확대할 범위
                            horizontal_flip=True, #랜덤하게 이미지 수평 뒤집기
                            fill_mode='nearest') #회전, 가로/세로 이동으로 인해 새롭게 생성해야 할 픽셀 채울 전략

In [15]:
#드롭아웃을 포함한 새로운 컨브넷 정의하기

model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
             optimizer=optimizers.RMSprop(lr=1e-4),
             metrics=['acc'])

In [16]:
#데이터 증식 제너레이터 사용해 컨브넷 훈련하기

train_datagen = ImageDataGeneratorGeneratorGeneratorGenerator(rescale = 1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255) #검증데이터는 증식되면 안됨

train_generator = train_datagen.flow_from_directory(train_dir,
                                                   target_size=(150,150),
                                                   batch_size=32,
                                                   class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                       target_size=(150,150),
                                                       batch_size=32,
                                                       class_mode='binary')

history = model.fit.generator(train_generator,
                             steps_per_epoch = 100,
                             epochs=100,
                             validation_data=validation_generator,
                             validation_steps=50)

#모델 저장
model.save('cats_and_dogs_small_2.h5')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


AttributeError: 'function' object has no attribute 'generator'

#### 5.3 사전 훈련된 컨브넷 사용하기

대규모 이미지 분류 문제 위해 대량의 데이터셋에서 미리 훈련되어 저장된 네트워크

사전 훈련된 네트워크 사용하는 방법
1. 특성 추출
2. 미세 조정


#### 5.3.1 특성 추출

사전에 학습된 네트워크 표현 사용해 새로운 샘플에서 흥미로운 특성 뽑아내는 것   
특성 사용해 새로운 분류기 처음부터 훈련

연속된 합성곱, 풀링 층으로 시작해 완전 연결 분류기로 끝남

합성곱 층만 재사용, 완전 연결 분류기는 일반적으로 재사용 X

특성 합성곱 층에서 추출한 표현의 일반성(과 재사용성) 수준은 모델에 있는 층의 깊이에 달려 있음   
모델의 하위 층은 (에지, 색, 질감 등) 지역적, 매우 일반적 특성 맵 추출함   
상위 층은 ('강아지 눈', '고양이 귀' 등) 추상적인 개념 추출   
-> 모델의 하위 층 몇 개만 특성 추출에 사용하는 것이 좋음

In [17]:
#VGG16 합성곱 기반 층 만들기

from keras.applications import VGG16

conv_base = VGG16(weights='imagenet', #모델 초기화할 가중치 체크포인트 지정
                 include_top=False, #네트워크 최상위 완전 연결 분류기 포함 여부 지정(기본값은 포함)
                 input_shape=(150,150,3)) #네트워크에 주입할 이미지 텐서의 크기, 선택 사항(지정 안하면 어떤 크기의 입력도 처리 가능)


58892288/58889256 [==============================] - 16s 0us/step


**데이터 증식 사용하지 않는 빠른 특성 추출**   

빠르고 비용 적게 듬, 데이터 증식 사용 불가능


In [ ]:
#사전 훈련된 합성곱 기반 층 사용한 특성 추출하기

import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = './datasets/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 20


def extract_features(directory, sample_count):
    features = np.zeros(shape =(sample_count, 4,4,512))
    labels=np.zeros(shape=(sample_count))
    generator=datagen.flow_from_directory(directory,
                                         target_size=(150,150),
                                         batch_size=batch_size,
                                         class_mode='binary')
    i=0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size : (i+1) * batch_size] = features_batch
        labels[i*batch_size : (i+1) * batch_size] = labels_batch
        i+=1
        if i * batch_size >=sample_count:
            break
            #제너레이터는 루프 안에서 무한하게 데이터 만들어 내서 모든 이미지 한 번씩 처리 후에는 중지함
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)